# This Notebook computes the ROIs from the spectrograms of the 60 second audio recordings.

In [15]:
# Import section


import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from PIL import Image, ImageDraw

In [16]:
# Extract the ROIs from the spectrograms. Also annotates existing spectrogram with bounding boxes of ROIs.

In [17]:
cwd=os.getcwd()

# root_dir of where to put annotated spectrograms, etc.
root_dir = cwd+'/ROIs/'

# Where the computed spectrograms are located.
spectrogram_dir = cwd+'/Spectrograms/'

# Where the ROI .csv files are located.
rois_dir = cwd+'/ROIs/'

# The .csv file with the info about the ROIs.
rois_info_file = cwd+'/ROIs/birds.csv'

# Read the ROI info.
rois_info = pd.read_csv(rois_info_file)

n_birds = 5;


In [18]:
# For each bird, read the ROI file and process the ROIs.
for i in range(n_birds):
    # Read ROI .csv file for this bird.
    roi_csv_file = rois_dir + rois_info['roi_file'][i]
    bird_rois_info = pd.read_csv(roi_csv_file);
    n_rois = rois_info['roi_count'][i]
    # Process each ROI for this bird.
    for j in range(n_rois):
        # Always get original spectroram.
        recording = bird_rois_info['recording'][j]
        spectrogram = recording + '.png'
        spectrogram_filename = spectrogram_dir + spectrogram
        spectrogram_image = plt.imread(spectrogram_filename)
        
        # Check to see if this spectrogram has already been marked with an ROI.
        spectrogram_roi = recording + '.roi.png'
        spectrogram_roi_filename = root_dir + rois_info['bird'][i] + '/' + spectrogram_roi
        
        if os.path.exists(spectrogram_roi_filename):
            # Read already marked up spectrogram.
            spectrogram_roi_image_PIL = Image.open(spectrogram_roi_filename);
        else:
            try:
                os.makedirs(root_dir + rois_info['bird'][i])
            except:
                pass
            spectrogram_roi_image_PIL = Image.open(spectrogram_filename);

        # Get size of spectrogram.
        n_rows = np.shape(spectrogram_image)[0]
        n_cols = np.shape(spectrogram_image)[1]
            
        # Get time and frequency coordinates of spectrogram from _info.txt file.
        spectrogram_info = recording + '_info.txt'
        spectrogram_info_filename = spectrogram_dir + spectrogram_info
        spectrograminfo_file = open(spectrogram_info_filename,'r')

        time_coords = np.zeros(n_cols)
        for x in range(n_cols):
            time_coords[x] = spectrograminfo_file.readline()
        freq_coords = np.zeros(n_rows)
        for x in range(n_rows):
            freq_coords[x] = spectrograminfo_file.readline()
        spectrograminfo_file.close()
        # Reverse.
        freq_coords = np.flip(freq_coords)
        
        # Find pixel bounds of ROI.
        top_left_row = np.where(freq_coords < bird_rois_info['y2'][j])
        bottom_right_row = np.where(freq_coords < bird_rois_info['y1'][j])
        top_left_col = np.where(time_coords > bird_rois_info['x1'][j])
        bottom_right_col = np.where(time_coords > bird_rois_info['x2'][j])
        width = bottom_right_col[0][0] - top_left_col[0][0] + 1;
        height = bottom_right_row[0][0] - top_left_row[0][0] + 1;
        
        # Draw rectangle.
        img1 = ImageDraw.Draw(spectrogram_roi_image_PIL)  
        img1.rectangle([(top_left_col[0][0],top_left_row[0][0]),(bottom_right_col[0][0],bottom_right_row[0][0])], outline ='white')
        
        # Save spectrogram with marked ROIs.
        spectrogram_roi_image_PIL.save(spectrogram_roi_filename)
        
        # Extract and save ROI.
        image_roi = spectrogram_image[top_left_row[0][0]-1:bottom_right_row[0][0]-1, top_left_col[0][0]-1:bottom_right_col[0][0]-1]
        image_roi_filename = root_dir+rois_info['bird'][i]+'/ROIs/'+rois_info['bird'][i]+'_'+str(bird_rois_info['id'][j])+'.png'
        
        if os.path.exists(root_dir+rois_info['bird'][i]+'/ROIs/'):
            plt.imsave(image_roi_filename, image_roi, cmap='gray')
        else:
            try:
                os.makedirs(root_dir+rois_info['bird'][i]+'/ROIs/')
            except:
                pass
            plt.imsave(image_roi_filename, image_roi, cmap='gray')